<a href="https://colab.research.google.com/github/anilss147/Image-transformations/blob/main/Multiclass_Image_classification_CNN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [ ]:
import os
import zipfile
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import os
import glob
import pandas as pd


In [ ]:

base_dir2 = '/content/drive/MyDrive/SACK_THURSDAY2'

train_dir2 = os.path.join(base_dir2, 'train')

validation_dir2 =os.path.join(base_dir2, 'validation')

train_arielblue_dir = os.path.join(train_dir2, 'ARIELRED')

train_ATIDEPLUS_dir = os.path.join(train_dir2, 'ATIDEPLUS')

train_BARIELBLUE_dir = os.path.join(train_dir2, 'BARIELBLUE')

train_CONVEYOR_dir = os.path.join(train_dir2, 'CONVEYOR')

train_TIDEYELLOW_dir = os.path.join(train_dir2, 'TIDEYELLOW')

train_UNKNOWN_dir = os.path.join(train_dir2, 'UNKNOWN')

train_ARIELBLUE_2000_dir = os.path.join(train_dir2, 'ARIELBLUE_2000')

train_TIDEPLUS_2000_dir = os.path.join(train_dir2, 'TIDEPLUS_2000')

train_TIDEYELLOW_2000_test_dilr = os.path.join(train_dir2, 'TIDEYELLOW_2000_test')

train_ARIELRED_2000_test_dilr = os.path.join(train_dir2, 'ARIELRED_2000')

validation_arielblue_dir = os.path.join(validation_dir2, 'ARIELRED')

validation_ATIDEPLUS_dir = os.path.join(validation_dir2, 'ATIDEPLUS')

validation_BARIELBLUE_dir = os.path.join(validation_dir2, 'BARIELBLUE')

validation_CONVEYOR_dir = os.path.join(validation_dir2, 'CONVEYOR')

validation_TIDEYELLOW_dir = os.path.join(validation_dir2, 'TIDEYELLOW')

validation_UNKNOWN_dir = os.path.join(validation_dir2, 'UNKNOWN')


In [ ]:
train_all = [train_arielblue_dir,train_ATIDEPLUS_dir,train_BARIELBLUE_dir,train_CONVEYOR_dir,train_ARIELRED_2000_test_dilr,train_TIDEYELLOW_dir,train_TIDEYELLOW_2000_test_dilr,train_UNKNOWN_dir,train_ARIELBLUE_2000_dir,train_TIDEPLUS_2000_dir]
validation_all = [validation_arielblue_dir,validation_ATIDEPLUS_dir,validation_BARIELBLUE_dir,validation_CONVEYOR_dir,validation_TIDEYELLOW_dir,validation_UNKNOWN_dir]
imag_name_list = []
for input_dir in train_all + validation_all:
  im_names = sorted(glob.glob(os.path.join(input_dir, '*.jpeg')))
  for i,im_name in enumerate(im_names):
    imag_name_list.append(im_name)
data = pd.DataFrame({"image_path":imag_name_list})
lbl = ['ARIELBLUE','TIDEYELLOW','ARIELRED','UNKNOWN','TIDEPLUS','CONVEYOR']#,'TIDEPLUS'
def label_fn(path):
 for i,l in enumerate(lbl):
    if l in path:
      return i
data["label"] = data.image_path.apply(lambda x: label_fn(x))
data.to_csv("train.csv",index=False)

In [ ]:
data.label.value_counts()

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model
import numpy as np


In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
# All images will be rescaled by 1./255
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range = 40, 
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   horizontal_flip = True, 
                                   brightness_range = (0.5, 1.5))
val_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range = 40, 
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   horizontal_flip = True, 
                                   brightness_range = (0.5, 1.5))


train = pd.read_csv("/content/train.csv")
xtrain,xtest,ytrain,ytest = train_test_split(train.image_path,train.label,test_size=0.1,shuffle = True )
train_df = pd.DataFrame({"img":xtrain.values,"label":ytrain.values.astype(np.str)})
val_df = pd.DataFrame({"img":xtest.values,"label":ytest.values.astype(np.str)})
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
        directory="",
        x_col="img",
        y_col = "label",  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')
 
# Flow validation images in batches of 20 using val_datagen generator
validation_generator = val_datagen.flow_from_dataframe(
        dataframe=val_df,
        x_col = "img",
        y_col = "label",
        directory="",
        target_size=(150, 150),
        batch_size=2,
        class_mode='categorical')

Found 24970 validated image filenames belonging to 6 classes.
Found 2775 validated image filenames belonging to 6 classes.


In [ ]:
train_generator.class_indices

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
import tensorflow as tf
import numpy as np
#import keras                                                            
with tf.device("/device:GPU:0"):
  cnn = tf.keras.models.Sequential()
  cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu', input_shape=[150, 150, 3]))
  cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
  cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu'))
  cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
  cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
  cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
  cnn.add(tf.keras.layers.Flatten())
  cnn.add(tf.keras.layers.Dense(128, activation='relu'))
  cnn.add(tf.keras.layers.Dense(64, activation='relu'))
  cnn.add(tf.keras.layers.Dense(6, activation='softmax'))
  cnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
  cnn.fit(x=train_generator, validation_data=validation_generator, epochs=20)

In [ ]:
cnn.history

In [ ]:
cnn.get_config()
#cnn.history

In [ ]:
np.mean(cnn.history['acc'])

In [ ]:
cnn.summary()

In [ ]:
t = "513"
export_path_keras1 = '/content/drive/MyDrive/NEWNETWORK/model/3 12'#+"/{}.h5".format(int(t))
export_path_keras = '/content/drive/MyDrive/NEWNETWORK/model/3 12'+"/{}.h5".format(int(t))
print(export_path_keras)
cnn.save(export_path_keras1)
cnn.save(export_path_keras)

class_map = train_generator.class_indices
  

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

path = '/content/drive/MyDrive/NEWNETWORK/model/3 12/513.h5'
reloaded = tf.keras.models.load_model(
  path, 
  # `custom_objects` tells keras how to load a `hub.KerasLayer`
  custom_objects={'KerasLayer': hub.KerasLayer})

reloaded.summary()


In [ ]:
import numpy as np
from keras_preprocessing import image

test_image = image.load_img("/content/drive/MyDrive/SACK_THURSDAY2/train/CONVEYOR/1(100).jpeg",target_size = (150,150))

test_image1 = image.img_to_array(test_image)
test_image2 = np.expand_dims(test_image1, axis =0)
result = reloaded.predict(test_image2)

print((result))
print((result[0][0]))
print((result[0][1]))
print((result[0][2]))
print((result[0][3]))
print((result[0][4]))
print((result[0][5]))
#lbl = ['ARIELBLUE','TIDEYELLOW','ARIELRED','UNKNOWN','TIDEPLUS','CONVEYOR']